# Demonstrate Transfer Learning and Tuning

Transfer Learning and Tuning demonstration on Cats and Dogs filtered dataset

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Library Imports

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm_notebook
from sklearn.metrics import classification_report, confusion_matrix

## Load the image dataset

In [6]:
PARENT_DIR = os.environ.get("ONE_DRIVE")
DATA_DIR = 'Datasets/LargeDatasets/cats_and_dogs_filtered'
DATA_DIR = os.path.join(PARENT_DIR, DATA_DIR)
print(DATA_DIR)

/Users/suvosmac/OneDrive/Datasets/LargeDatasets/cats_and_dogs_filtered


In [10]:
# Define the training and validation paths

train_dir = os.path.join(DATA_DIR, 'train')
validation_dir = os.path.join(DATA_DIR, 'validation')

# Directory for training images
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory for validation images
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Check the first five names of the cats ang dogs
cats_fnames = os.listdir(train_cats_dir)
dogs_fnames = os.listdir(validation_dogs_dir)

print(f"First 5 names of the cats training images - {cats_fnames[:5]}")
print(f"First 5 names of the dogs training images - {dogs_fnames[:5]}")

print(f"Total Training Cats Images - {len(os.listdir(train_cats_dir))}")
print(f"Total training dogs Images - {len(os.listdir(train_dogs_dir))}")
print(f"Total validation cats Images - {len(os.listdir(validation_cats_dir))}")
print(f"Total validation dogs Images - {len(os.listdir(validation_dogs_dir))}")

First 5 names of the cats training images - ['cat.952.jpg', 'cat.946.jpg', 'cat.6.jpg', 'cat.749.jpg', 'cat.991.jpg']
First 5 names of the dogs training images - ['dog.2266.jpg', 'dog.2272.jpg', 'dog.2299.jpg', 'dog.2058.jpg', 'dog.2064.jpg']
Total Training Cats Images - 1000
Total training dogs Images - 1000
Total validation cats Images - 500
Total validation dogs Images - 500


### Build the Image Data Generators

In [11]:
# we will preprocess the image, before we will apply the image to model for training
# Adding data augmentation parameters

train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

# Note that validation data should not be augmented
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Flow training images in the batches of 20 using train datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=20, class_mode='binary', target_size=(150, 150))

# Flow validation images in the batches of 20 using test datagen generator
validation_generator = test_datagen.flow_from_directory(validation_dir, batch_size=20, class_mode='binary', target_size=(150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
